In [1]:
import pandas as pd

# Load the newly uploaded files into DataFrames - from Quickbooks
avid_expense_1 = pd.read_csv("AvidExpenses.csv")
avid_expense_2 = pd.read_csv("AvidExpenses2.csv")

# Display the first few rows of both datasets to understand their structure
avid_expense_1_head = avid_expense_1.head()
avid_expense_2_head = avid_expense_2.head()

avid_expense_1_head, avid_expense_2_head


(            Natural Electric LLC                     Unnamed: 1  Unnamed: 2  \
 0             Transaction Report                            NaN         NaN   
 1                      All Dates                            NaN         NaN   
 2                            NaN                        Account        Date   
 3  51400 Job Materials Purchased                            NaN         NaN   
 4                            NaN  51400 Job Materials Purchased  10/24/2019   
 
          Unnamed: 3 Unnamed: 4  
 0               NaN        NaN  
 1               NaN        NaN  
 2  Transaction type     Amount  
 3               NaN        NaN  
 4           Expense      $9.12  ,
                   Natural Electric LLC                           Unnamed: 1  \
 0                   Transaction Report                                  NaN   
 1                            All Dates                                  NaN   
 2                                  NaN                              Acco

In [2]:
# Reload the datasets, skipping irrelevant header rows
avid_expense_1_clean = pd.read_csv("AvidExpenses.csv", skiprows=4)
avid_expense_2_clean = pd.read_csv("AvidExpenses2.csv", skiprows=4)

# Ensure the columns align between both datasets
columns_match = avid_expense_1_clean.columns == avid_expense_2_clean.columns

# Concatenate the datasets if columns align
if columns_match.all():
    combined_avid_expenses = pd.concat([avid_expense_1_clean, avid_expense_2_clean], ignore_index=True)
else:
    combined_avid_expenses = None  # Handle misalignment scenario

# Display the first few rows of the combined dataset or a message if the columns don't align
if combined_avid_expenses is not None:
    print(combined_avid_expenses.head())
else:
    print("Columns do not align between the two datasets.")


                      Unnamed: 0                        Account        Date  \
0  51400 Job Materials Purchased                            NaN         NaN   
1                            NaN  51400 Job Materials Purchased  10/24/2019   
2                            NaN  51400 Job Materials Purchased  12/20/2019   
3                            NaN  51400 Job Materials Purchased  12/27/2019   
4                            NaN  51400 Job Materials Purchased  12/27/2019   

  Transaction type     Amount  
0              NaN        NaN  
1          Expense      $9.12  
2             Bill  $1,087.81  
3    Vendor Credit    -$14.93  
4             Bill    $991.06  


In [3]:
# Remove the first column
avid_expenses_cleaned = combined_avid_expenses.iloc[:, 1:]

# Drop rows with any NaN values
avid_expenses_cleaned = avid_expenses_cleaned.dropna()

# Reset the index after dropping rows
avid_expenses_cleaned.reset_index(drop=True, inplace=True)

# Display the cleaned dataset to confirm the changes
avid_expenses_cleaned.head()

,Account,Date,Transaction type,Amount
0,51400 Job Materials Purchased,10/24/2019,Expense,$9.12
1,51400 Job Materials Purchased,12/20/2019,Bill,"$1,087.81"
2,51400 Job Materials Purchased,12/27/2019,Vendor Credit,-$14.93
3,51400 Job Materials Purchased,12/27/2019,Bill,$991.06
4,51400 Job Materials Purchased,12/27/2019,Bill,$52.16


In [4]:
#check to be sure rows with missing values were removed
# Identify rows with any missing values
rows_with_missing_values = avid_expenses_cleaned[avid_expenses_cleaned.isna().any(axis=1)]

# Display the rows with missing values (if any)
if not rows_with_missing_values.empty:
    print("Rows with missing values:")
    print(rows_with_missing_values)
else:
    print("No rows with missing values found.")

No rows with missing values found.


In [5]:
# Display the data types of each column in the cleaned dataset
data_types_avid_expenses = avid_expenses_cleaned.dtypes

data_types_avid_expenses


Account             object
Date                object
Transaction type    object
Amount              object
dtype: object

In [6]:
# Convert 'Amount' column to numeric by removing currency symbols and commas
avid_expenses_cleaned['Amount'] = (
    avid_expenses_cleaned['Amount']
    .replace(r'[\$,]', '', regex=True)
    .astype(float)
)

# Convert 'Date' column to datetime format
avid_expenses_cleaned['Date'] = pd.to_datetime(
    avid_expenses_cleaned['Date'], errors='coerce'
)

# Display the first few rows to confirm the changes
avid_expenses_cleaned.head()


,Account,Date,Transaction type,Amount
0,51400 Job Materials Purchased,2019-10-24,Expense,9.12
1,51400 Job Materials Purchased,2019-12-20,Bill,1087.81
2,51400 Job Materials Purchased,2019-12-27,Vendor Credit,-14.93
3,51400 Job Materials Purchased,2019-12-27,Bill,991.06
4,51400 Job Materials Purchased,2019-12-27,Bill,52.16


In [7]:
# Display all unique values in the 'Account' column
unique_accounts_avid = avid_expenses_cleaned['Account'].unique()

unique_accounts_avid


array(['51400 Job Materials Purchased', '50800 Equipment Rental for Jobs',
       '53600 Subcontractors Expense', 'PT Casual Labor',
       '52000 Other Job Related Costs',
       '61000 Business Licenses and Permits', 'Fuel', 'Employee Expense',
       '64900 Office Supplies', '67200 Repairs and Maintenance',
       'Legal Fees', 'Wages', 'Taxes'], dtype=object)

In [8]:
# Remove leading numbers and spaces from the 'Account' column using regex
avid_expenses_cleaned['Account'] = avid_expenses_cleaned['Account'].str.replace(r'^\d+\s+', '', regex=True)

# Display the unique account values after cleaning
unique_accounts_cleaned = avid_expenses_cleaned['Account'].unique()

unique_accounts_cleaned


array(['Job Materials Purchased', 'Equipment Rental for Jobs',
       'Subcontractors Expense', 'PT Casual Labor',
       'Other Job Related Costs', 'Business Licenses and Permits', 'Fuel',
       'Employee Expense', 'Office Supplies', 'Repairs and Maintenance',
       'Legal Fees', 'Wages', 'Taxes'], dtype=object)

In [9]:
# Replace specified account names with 'Labor'
avid_expenses_cleaned['Account'] = avid_expenses_cleaned['Account'].replace(
    ['PT Casual Labor', 'Wages', 'Taxes'], 'Labor'
)

# Display the unique account values to confirm the changes
unique_accounts_updated = avid_expenses_cleaned['Account'].unique()

unique_accounts_updated


array(['Job Materials Purchased', 'Equipment Rental for Jobs',
       'Subcontractors Expense', 'Labor', 'Other Job Related Costs',
       'Business Licenses and Permits', 'Fuel', 'Employee Expense',
       'Office Supplies', 'Repairs and Maintenance', 'Legal Fees'],
      dtype=object)

In [10]:
# Remove the row with 'Legal Fees'
avid_expenses_cleaned = avid_expenses_cleaned[avid_expenses_cleaned['Account'] != 'Legal Fees']

# Display the unique account types to confirm the change
unique_account_types_final = avid_expenses_cleaned['Account'].unique()

unique_account_types_final

array(['Job Materials Purchased', 'Equipment Rental for Jobs',
       'Subcontractors Expense', 'Labor', 'Other Job Related Costs',
       'Business Licenses and Permits', 'Fuel', 'Employee Expense',
       'Office Supplies', 'Repairs and Maintenance'], dtype=object)

In [11]:
# Replace specified account names with 'Other'
avid_expenses_cleaned['Account'] = avid_expenses_cleaned['Account'].replace(
    ['Other Job Related Costs', 'Business Licenses and Permits', 
     'Employee Expense', 'Office Supplies', 
     'Repairs and Maintenance'], 'Other'
)

# Display the unique account values to confirm the changes
unique_accounts_final = avid_expenses_cleaned['Account'].unique()

unique_accounts_final


array(['Job Materials Purchased', 'Equipment Rental for Jobs',
       'Subcontractors Expense', 'Labor', 'Other', 'Fuel'], dtype=object)

In [12]:
#adjust dates to more accurately reflect when labor was on job instead of billing and paycheck dates
# Subtract one week from the 'Date' column for 'Labor' and 'Subcontractors Expense' entries
avid_expenses_cleaned.loc[
    avid_expenses_cleaned['Account'].isin(['Labor', 'Subcontractors Expense']), 'Date'
] = avid_expenses_cleaned.loc[
    avid_expenses_cleaned['Account'].isin(['Labor', 'Subcontractors Expense']), 'Date'
] - pd.Timedelta(weeks=1)

# Display the updated entries for 'Labor' and 'Subcontractors Expense' to confirm the changes
updated_entries = avid_expenses_cleaned[
    avid_expenses_cleaned['Account'].isin(['Labor', 'Subcontractors Expense'])
]

updated_entries


,Account,Date,Transaction type,Amount
458,Subcontractors Expense,2020-05-31,Bill,1548.00
459,Subcontractors Expense,2020-06-07,Bill,2293.14
460,Subcontractors Expense,2020-06-07,Bill,959.44
461,Subcontractors Expense,2021-04-02,Bill,3185.00
462,Subcontractors Expense,2021-04-09,Bill,5390.00
...,...,...,...,...
1202,Labor,2021-09-24,Payroll Check,56.60
1203,Labor,2021-10-08,Payroll Check,9.68
1204,Labor,2021-10-08,Payroll Check,42.27
1205,Labor,2021-10-08,Payroll Check,5.05


In [13]:
# Remove all rows where the 'Date' is later than October 5th, 2021
avid_expenses_cleaned = avid_expenses_cleaned[avid_expenses_cleaned['Date'] <= '2021-10-05']

# Reset the index after removing rows
avid_expenses_cleaned.reset_index(drop=True, inplace=True)

# Display the cleaned dataset to confirm the removal
avid_expenses_cleaned


,Account,Date,Transaction type,Amount
0,Job Materials Purchased,2019-10-24,Expense,9.12
1,Job Materials Purchased,2019-12-20,Bill,1087.81
2,Job Materials Purchased,2019-12-27,Vendor Credit,-14.93
3,Job Materials Purchased,2019-12-27,Bill,991.06
4,Job Materials Purchased,2019-12-27,Bill,52.16
...,...,...,...,...
1191,Labor,2021-09-10,Payroll Check,55.78
1192,Labor,2021-09-24,Payroll Check,145.65
1193,Labor,2021-09-24,Payroll Check,27.08
1194,Labor,2021-09-24,Payroll Check,53.48


In [14]:
# Rename the DataFrame to 'avid_expense_df'
avid_expense_df = avid_expenses_cleaned

# Save the DataFrame as 'AvidExpensesFinal.csv'
csv_path = "AvidExpensesFinal.csv"
avid_expense_df.to_csv(csv_path, index=False)

# Confirm the operation
csv_path


'AvidExpensesFinal.csv'

In [15]:
#import NEJobs CSV to add Avid data
NE_jobs_df = pd.read_csv("NEJobs.csv")
NE_jobs_df

,Job Name,Materials,Labor,Equipment Rental,Subcontractor Expense,Fuel,Other,Total Cost,Total Income,Profit Margin,Vendor Credits
0,The Manor,246049.05,199060.77,4181.55,47900.54,6010.92,26252.18,529455.01,828476.88,0.360000,-30620.53
1,Clewiston,97240.76,123064.47,23202.76,0.00,2434.42,1290.76,247233.17,374819.47,0.340394,-825.01
2,Avid,292934.72,226812.45,18930.11,105170.08,5916.69,1101.21,650865.26,691814.50,0.070192,-7667.20


In [16]:
# Remove the row where 'Total Income' is 691814.50 (I added this unintentionally with wrong amount)
NE_jobs_df= NE_jobs_df[NE_jobs_df['Total Income'] != 691814.50]

# Reset the index after removing the row
NE_jobs_df.reset_index(drop=True, inplace=True)

# Save the cleaned DataFrame back to the CSV
csv_path = "NEjobs.csv"
NE_jobs_df.to_csv(csv_path, index=False)

# Display the cleaned DataFrame to confirm the removal
NE_jobs_df


,Job Name,Materials,Labor,Equipment Rental,Subcontractor Expense,Fuel,Other,Total Cost,Total Income,Profit Margin,Vendor Credits
0,The Manor,246049.05,199060.77,4181.55,47900.54,6010.92,26252.18,529455.01,828476.88,0.360000,-30620.53
1,Clewiston,97240.76,123064.47,23202.76,0.00,2434.42,1290.76,247233.17,374819.47,0.340394,-825.01


In [17]:
# Create the new row for the Avid job with all calculated values
avid_row = {
    "Job Name": "Avid",
    "Materials": avid_expense_df[avid_expense_df['Account'] == 'Job Materials Purchased']['Amount'].sum(),
    "Labor": avid_expense_df[avid_expense_df['Account'] == 'Labor']['Amount'].sum(),
    "Equipment Rental": avid_expense_df[avid_expense_df['Account'] == 'Equipment Rental for Jobs']['Amount'].sum(),
    "Subcontractor Expense": avid_expense_df[avid_expense_df['Account'] == 'Subcontractors Expense']['Amount'].sum(),
    "Fuel": avid_expense_df[avid_expense_df['Account'] == 'Fuel']['Amount'].sum(),
    "Other": avid_expense_df[avid_expense_df['Account'] == 'Other']['Amount'].sum(),
    "Total Cost": avid_expense_df['Amount'].sum(),
    "Total Income": 741244.95,  
    "Profit Margin": (741244.95 - avid_expense_df['Amount'].sum()) / 741244.95,
    "Vendor Credits": avid_expense_df[avid_expense_df['Transaction type'] == 'Vendor Credit']['Amount'].sum()
}


# Append the Avid row to NE_jobs_df
NE_jobs_df = pd.concat([NE_jobs_df, pd.DataFrame([avid_row])], ignore_index=True)

# Save the updated NE_jobs_df to the CSV file
csv_path = "NEjobs.csv"
NE_jobs_df.to_csv(csv_path, index=False)

# Display the updated NE_jobs_df to confirm
NE_jobs_df

,Job Name,Materials,Labor,Equipment Rental,Subcontractor Expense,Fuel,Other,Total Cost,Total Income,Profit Margin,Vendor Credits
0,The Manor,246049.05,199060.77,4181.55,47900.54,6010.92,26252.18,529455.01,828476.88,0.360000,-30620.53
1,Clewiston,97240.76,123064.47,23202.76,0.00,2434.42,1290.76,247233.17,374819.47,0.340394,-825.01
2,Avid,304377.83,239664.68,18930.11,134450.08,6104.87,1101.21,704628.78,741244.95,0.049398,-7727.31


In [19]:
NE_jobs_df.to_excel('NEJobs.xlsx', index=False)
avid_expense_df.to_excel('AvidExpensesFinal.xlsx', index=False)